In [34]:
import pandas as pd
!pip install tabulate

# Load the dataset
df = pd.read_csv('/content/Myntra_fashion_products.csv')

# Check the first few rows
print(df.head())

                                                name       sku       mpn  \
0  DKNY Unisex Black & Grey Printed Medium Trolle...  10017413  10017413   
1  EthnoVogue Women Beige & Grey Made to Measure ...  10016283  10016283   
2  SPYKAR Women Pink Alexa Super Skinny Fit High-...  10009781  10009781   
3  Raymond Men Blue Self-Design Single-Breasted B...  10015921  10015921   
4  Parx Men Brown & Off-White Slim Fit Printed Ca...  10017833  10017833   

   price  in_stock currency       brand  \
0  11745      True      INR        DKNY   
1   5810      True      INR  EthnoVogue   
2    899      True      INR      SPYKAR   
3   5599      True      INR     Raymond   
4    759      True      INR        Parx   

                                         description  \
0  Black and grey printed medium trolley bag, sec...   
1  Beige & Grey made to measure kurta with churid...   
2  Pink coloured wash 5-pocket high-rise cropped ...   
3  Blue self-design bandhgala suitBlue self-desig...   
4  B

In [35]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset
df = pd.read_csv('/content/Myntra_fashion_products.csv')

# Data Preprocessing
df.loc[:, 'price'] = pd.to_numeric(df['price'], errors='coerce')
df = df.dropna(subset=['description', 'price', 'brand', 'gender'])
df.reset_index(drop=True, inplace=True)

# TF-IDF Vectorizer for product descriptions
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['description'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to get user preferences
def get_user_preferences():
    print("Please enter your preferences:")
    brand = input("Enter the preferred brand (or leave blank to skip): ")
    gender = input("Enter the gender preference (Men/Women/Unisex): ")
    min_price = float(input("Enter minimum price: "))
    max_price = float(input("Enter maximum price: "))

    filtered_df = df[(df['price'] >= min_price) & (df['price'] <= max_price)]

    if brand:
        filtered_df = filtered_df[filtered_df['brand'].str.contains(brand, case=False)]
    if gender:
        filtered_df = filtered_df[filtered_df['gender'].str.contains(gender, case=False)]

    return filtered_df

# Function to recommend products based on cosine similarity
def recommend_product(product_idx, cosine_sim=cosine_sim, num_recommendations=5):
    sim_scores = list(enumerate(cosine_sim[product_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_recommendations+1]
    product_indices = [x[0] for x in sim_scores]
    return df.iloc[product_indices][['name', 'brand', 'price', 'description', 'gender']]

In [36]:
def get_user_preferences():
    print("Please enter your preferences:")

    brand = input("Enter the preferred brand (or leave blank to skip): ").strip()
    gender = input("Enter the gender preference (Men/Women/Unisex): ").strip().lower()
    min_price = float(input("Enter minimum price: "))
    max_price = float(input("Enter maximum price: "))
    clothing_type = input("Enter the type of clothing you're looking for (e.g., shoes, shirts, kurtas, pants, etc.): ").strip().lower()
    occasion = input("Enter the occasion you're shopping for (e.g., casual, formal, party, etc.): ").strip().lower()

    # Initial strict filtering
    filtered_df = df[
        (df['price'] >= min_price) &
        (df['price'] <= max_price)
    ]
    if brand:
        filtered_df = filtered_df[filtered_df['brand'].str.contains(brand, case=False)]
    if gender:
        filtered_df = filtered_df[filtered_df['gender'].str.lower() == gender]
    if clothing_type:
        filtered_df = filtered_df[filtered_df['description'].str.contains(clothing_type, case=False)]
    if occasion:
        filtered_df = filtered_df[filtered_df['description'].str.contains(occasion, case=False)]

    if not filtered_df.empty:
        return filtered_df, True  # True = exact match

    # Fallback - loosen filters for suggestions
    suggestion_df = df.copy()
    if brand:
        suggestion_df = suggestion_df[suggestion_df['brand'].str.contains(brand, case=False)]
    if gender:
        suggestion_df = suggestion_df[suggestion_df['gender'].str.lower() == gender]
    if clothing_type:
        suggestion_df = suggestion_df[suggestion_df['description'].str.contains(clothing_type, case=False)]

    suggestion_df = suggestion_df.sort_values(by='price').head(10)  # Suggest top 10 by price
    return suggestion_df, False  # False = suggestions



In [37]:
# Run preference gathering
user_filtered_df, is_strict_match = get_user_preferences()

if not user_filtered_df.empty:
    table = tabulate(
        user_filtered_df[['name', 'brand', 'price', 'gender', 'description']],
        headers='keys', tablefmt='pipe', showindex=False
    )
    if is_strict_match:
        print("\n🎯 Filtered Products based on your preferences:")
    else:
        print("\n🔍 No exact match found. Here are some suggestions you might like:")
    print(table)
else:
    print("Sorry! Couldn't find any products or suggestions at the moment.")


Please enter your preferences:
Enter the preferred brand (or leave blank to skip): 
Enter the gender preference (Men/Women/Unisex): MEN
Enter minimum price: 511
Enter maximum price: 56
Enter the type of clothing you're looking for (e.g., shoes, shirts, kurtas, pants, etc.): shirts
Enter the occasion you're shopping for (e.g., casual, formal, party, etc.): casual

🔍 No exact match found. Here are some suggestions you might like:
| name                                  | brand   |   price | gender   | description                                                                                                                                     |
|:--------------------------------------|:--------|--------:|:---------|:------------------------------------------------------------------------------------------------------------------------------------------------|
| UCLA Men Pack of 3 Round Neck T-shirt | UCLA    |    1299 | Men      | Pack of three T-shirtsTwo White and Grey striped T-shirt,